In [25]:
import config_cat_embedding
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, classification_report, roc_auc_score
from tqdm import tqdm_notebook as tqdm

from data_prep import bank_data_prep, adult_data_prep
from embedding_helper import create_network
import time
import random
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone


data_path = config_cat_embedding.paths['data']
data_path_out = config_cat_embedding.paths['data_output']
bank_data = pd.read_csv(data_path+'adult.csv', sep=',')
df_bank, cat_cols = adult_data_prep(bank_data)

X = df_bank.iloc[:, :-1]
target = df_bank.y

X_train, X_test, y_train, y_test = train_test_split(X, target, test_size=0.2, random_state=1500)

# label-hot encoding
import category_encoders as ce
label_encoder = ce.OrdinalEncoder(cols=cat_cols) 
X_train_ohe = label_encoder.fit_transform(X_train)
X_test_ohe = label_encoder.transform(X_test)  # Use transform() instead of fit_transform()

# Standard scaling
stc = StandardScaler()
X_train_scaled = stc.fit_transform(X_train_ohe)
X_test_scaled = stc.transform(X_test_ohe)


# Define the classifiers
seed = 42
number_of_features = X_train_scaled.shape[1]  # Number of features in the input data

models = [
    ('LR', LogisticRegression(solver='lbfgs', random_state=seed, max_iter=1000)),
    ('DT', DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=seed)),
    ('RF', RandomForestClassifier(n_estimators=200, max_depth=5, random_state=seed, min_samples_leaf=3)),
    ('KNN', KNeighborsClassifier(n_neighbors=3)),
    ('XGB', XGBClassifier(eval_metric='logloss')),
    ('SVM', SVC(gamma='scale', random_state=seed, probability=True)),
    ('MLP', KerasClassifier(build_fn=create_network, number_of_features=number_of_features, epochs=100, batch_size=100, verbose=0))
]



from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

# Define number of runs for cross-validation
n_runs = 5
kf = StratifiedKFold(n_splits=n_runs, shuffle=True, random_state=seed)

# Initialize dictionary to store results
results = {}

# Loop over the models
for name, model in models:
    print(f"Classifier: {name}")
    
    # Lists to store metrics for each fold
    accuracy_scores = []
    precision_scores = []
    recall_scores = []
    f1_scores = []
    roc_auc_scores = []
    computation_times = []
    
    # Perform cross-validation
    for train_idx, test_idx in kf.split(X_train_scaled, y_train):
        # Clone the model to ensure fresh training for each fold
        clf = clone(model)
        
        X_train_fold, X_test_fold = X_train_scaled[train_idx], X_train_scaled[test_idx]
        y_train_fold, y_test_fold = y_train.iloc[train_idx], y_train.iloc[test_idx]
        
        start_time = time.time()  # Start time
        
        clf.fit(X_train_fold, y_train_fold)
        y_pred = clf.predict(X_test_fold)#X_test_scaled
        y_pred_prob = clf.predict_proba(X_test_fold)[:, 1]  # Probabilities for ROC AUC
        
        end_time = time.time()  # End time
        computation_time = end_time - start_time  # Computation time
        
        # Append metrics for this fold
        accuracy_scores.append(accuracy_score(y_test_fold, y_pred))
        precision_scores.append(precision_score(y_test_fold, y_pred))
        recall_scores.append(recall_score(y_test_fold, y_pred))
        f1_scores.append(f1_score(y_test_fold, y_pred))
        roc_auc_scores.append(roc_auc_score(y_test_fold, y_pred_prob))
        computation_times.append(computation_time)
    
    # Compute mean and standard deviation of all metrics
    mean_accuracy = np.mean(accuracy_scores)
    std_accuracy = np.std(accuracy_scores)
    
    mean_precision = np.mean(precision_scores)
    std_precision = np.std(precision_scores)
    
    mean_recall = np.mean(recall_scores)
    std_recall = np.std(recall_scores)
    
    mean_f1 = np.mean(f1_scores)
    std_f1 = np.std(f1_scores)
    
    mean_roc_auc = np.mean(roc_auc_scores)
    std_roc_auc = np.std(roc_auc_scores)
    
    mean_time = np.mean(computation_times)
    std_time = np.std(computation_times)
    
    # Store results for the current model
    results[name] = {
        "Mean Accuracy": mean_accuracy, "STD Accuracy": std_accuracy,
        "Mean Precision": mean_precision, "STD Precision": std_precision,
        "Mean Recall": mean_recall, "STD Recall": std_recall,
        "Mean F1 Score": mean_f1, "STD F1 Score": std_f1,
        "Mean ROC AUC": mean_roc_auc, "STD ROC AUC": std_roc_auc,
        "Mean Time": mean_time, "STD Time": std_time
    }
    
    # Print results
    print(f"Mean Accuracy: {mean_accuracy:.3f} ± {std_accuracy:.3f}")
    print(f"Mean Precision: {mean_precision:.3f} ± {std_precision:.3f}")
    print(f"Mean Recall: {mean_recall:.3f} ± {std_recall:.3f}")
    print(f"Mean F1 Score: {mean_f1:.3f} ± {std_f1:.3f}")
    print(f"Mean ROC AUC: {mean_roc_auc:.3f} ± {std_roc_auc:.3f}")
    print(f"Mean Computation Time: {mean_time:.3f} ± {std_time:.3f} seconds")
    print()

# At the end, you can access the `results` dictionary for all the models


Classifier: LR
Mean Accuracy: 0.804 ± 0.004
Mean Precision: 0.688 ± 0.017
Mean Recall: 0.384 ± 0.013
Mean F1 Score: 0.493 ± 0.012
Mean ROC AUC: 0.855 ± 0.004
Mean Computation Time: 0.026 ± 0.001 seconds

Classifier: DT
Mean Accuracy: 0.815 ± 0.003
Mean Precision: 0.945 ± 0.013
Mean Recall: 0.270 ± 0.011
Mean F1 Score: 0.420 ± 0.014
Mean ROC AUC: 0.836 ± 0.005
Mean Computation Time: 0.038 ± 0.005 seconds

Classifier: RF
Mean Accuracy: 0.835 ± 0.005
Mean Precision: 0.868 ± 0.017
Mean Recall: 0.397 ± 0.020
Mean F1 Score: 0.544 ± 0.020
Mean ROC AUC: 0.893 ± 0.005
Mean Computation Time: 2.511 ± 0.333 seconds

Classifier: KNN
Mean Accuracy: 0.807 ± 0.005
Mean Precision: 0.621 ± 0.008
Mean Recall: 0.569 ± 0.018
Mean F1 Score: 0.593 ± 0.013
Mean ROC AUC: 0.809 ± 0.007
Mean Computation Time: 6.309 ± 1.639 seconds

Classifier: XGB
Mean Accuracy: 0.868 ± 0.006
Mean Precision: 0.776 ± 0.010
Mean Recall: 0.659 ± 0.018
Mean F1 Score: 0.713 ± 0.015
Mean ROC AUC: 0.925 ± 0.003
Mean Computation Time: 0

C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
C:\Users\P70071469\AppData\Local\anaconda3\envs\SLR\lib\site-packages\keras\src\layers\core\dense.py:87

Mean Accuracy: 0.830 ± 0.003
Mean Precision: 0.685 ± 0.022
Mean Recall: 0.592 ± 0.063
Mean F1 Score: 0.632 ± 0.027
Mean ROC AUC: 0.885 ± 0.004
Mean Computation Time: 69.639 ± 8.313 seconds

